In [1]:
import tensorflow as tf
from tensorflow.python.ops.nn_impl import _compute_sampled_logits
import numpy as np
import scipy


EPSILON = 1e-40


def tf_gumbel_keys(w):
    # sample some gumbels
    uniform = tf.random_uniform(
        tf.shape(w),
        minval=EPSILON,
        maxval=1.0)
    z = tf.log(-tf.log(uniform))
    w = w + z
    return w


def tf_continuous_topk(w, k, t, separate=False):
    khot_list = []
    onehot_approx = tf.zeros_like(w, dtype=tf.float32)
    for i in range(k):
        khot_mask = tf.maximum(1.0 - onehot_approx, EPSILON)
        w += tf.log(khot_mask)
        onehot_approx = tf.nn.softmax(w / t, axis=-1)
        khot_list.append(onehot_approx)
    if separate:
        return khot_list
    else:
        return tf.reduce_sum(khot_list, 0)


def tf_sample_subset(w, k, t=0.1):
    '''
    Args:
        w (Tensor): Float Tensor of weights for each element. In gumbel mode
            these are interpreted as log probabilities
        k (int): number of elements in the subset sample
        t (float): temperature of the softmax
    '''
    w = tf_gumbel_keys(w)
    return tf_continuous_topk(w, k, t)

###############################################


/home/nabeel/anaconda3/envs/nyu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nabeel/anaconda3/envs/nyu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nabeel/anaconda3/envs/nyu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nabeel/anaconda3/envs/nyu/lib/python3.7/site-pack

In [2]:
w = np.random.normal(size = [2, 50])

In [9]:
k = 25

w1 = tf.convert_to_tensor(w, dtype=tf.dtypes.float32)

t = 1

In [10]:
with tf.Session() as sess:
    #print(sess.run(w))
    subset = tf_continuous_topk(w1, k = k, t = t)
    print("Subset")
    print(sess.run(subset))
    #print(sess.run(subset > 1).sum())
    #print(sess.run(tf.boolean_mask(subset, subset > 1)))
    #print(sess.run(tf.nn.softmax(w1, axis = -1)))

Subset
[[0.4403324  0.16990203 0.6559786  0.14312331 0.3081524  0.0879445
  0.36700457 0.161529   0.4374837  0.30227256 0.7072869  0.26918152
  1.1540858  0.42025292 0.33912638 0.722536   0.763735   0.5183832
  0.3932995  0.15582824 0.45401567 1.0608776  0.41605785 0.42299742
  0.21457537 0.1441685  1.4688622  0.31164953 0.6264712  0.62124544
  0.7537335  0.42294872 0.1300711  0.80285937 0.8444582  0.30594316
  0.3438148  0.36532348 0.32563826 0.78325486 0.4722293  0.18426119
  0.17786753 0.89492583 0.13282962 1.0553182  0.17562264 0.9230698
  0.67827696 0.96919394]
 [0.28960466 0.20875862 0.23969477 0.36101398 0.36397654 0.20510592
  0.354275   0.30762753 0.35305923 0.5344066  0.23507136 0.65167207
  0.07046349 0.23391216 1.2257142  0.6029215  0.47052777 0.22190748
  0.14615043 0.47231734 0.19442178 0.55280626 0.22379097 0.078577
  0.32880104 0.53697336 0.58603126 0.7132288  0.42246002 0.5733961
  1.4231558  1.24103    0.12390844 1.2005006  0.17812574 0.2419317
  0.57470435 0.298784  

In [11]:
from utils import *
import torch
import torch.nn.functional as F

In [12]:
#F.softmax(torch.Tensor(w), dim = -1)

In [13]:
continuous_topk(torch.Tensor(w), k = k, t = t)

tensor([[0.4403, 0.1699, 0.6560, 0.1431, 0.3082, 0.0879, 0.3670, 0.1615, 0.4375,
         0.3023, 0.7073, 0.2692, 1.1541, 0.4203, 0.3391, 0.7225, 0.7637, 0.5184,
         0.3933, 0.1558, 0.4540, 1.0609, 0.4161, 0.4230, 0.2146, 0.1442, 1.4689,
         0.3116, 0.6265, 0.6212, 0.7537, 0.4229, 0.1301, 0.8029, 0.8445, 0.3059,
         0.3438, 0.3653, 0.3256, 0.7833, 0.4722, 0.1843, 0.1779, 0.8949, 0.1328,
         1.0553, 0.1756, 0.9231, 0.6783, 0.9692],
        [0.2896, 0.2088, 0.2397, 0.3610, 0.3640, 0.2051, 0.3543, 0.3076, 0.3531,
         0.5344, 0.2351, 0.6517, 0.0705, 0.2339, 1.2257, 0.6029, 0.4705, 0.2219,
         0.1462, 0.4723, 0.1944, 0.5528, 0.2238, 0.0786, 0.3288, 0.5370, 0.5860,
         0.7132, 0.4225, 0.5734, 1.4232, 1.2410, 0.1239, 1.2005, 0.1781, 0.2419,
         0.5747, 0.2988, 0.6895, 1.1708, 0.0498, 0.4040, 0.2399, 1.2897, 0.7791,
         1.1073, 0.5941, 0.9419, 0.4165, 0.2765]])